In [2]:
import pyspark
sc = pyspark.SparkContext('local[*]')

!rm -rf metastore_db/*.lck

from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

### Step 1
- Load the train and test sets
- Check the schema, the variables have their right types?
- If not, how to correctly load the datasets?

In [3]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

titanicSchemaTrain = StructType([StructField("PassengerId", IntegerType(), True),
                           StructField("Survived", IntegerType(), True),
                           StructField("Pclass",  IntegerType(), True), 
                           StructField("Name",  StringType(), True), 
                           StructField("Sex",  StringType(), True), 
                           StructField("Age",  FloatType(), True), 
                           StructField("SibSp",  IntegerType(), True), 
                           StructField("Parch",  IntegerType(), True), 
                           StructField("Ticket",  StringType(), True), 
                            StructField("Fare",  FloatType(), True), 
                            StructField("Cabin",  StringType(), True),
                           StructField("Embarked",  StringType(), True)]     
                          )

titanicSchemaTest = StructType([StructField("PassengerId", IntegerType(), True),
                           StructField("Pclass",  IntegerType(), True), 
                           StructField("Name",  StringType(), True), 
                           StructField("Sex",  StringType(), True), 
                           StructField("Age",  FloatType(), True), 
                           StructField("SibSp",  IntegerType(), True), 
                           StructField("Parch",  IntegerType(), True), 
                           StructField("Ticket",  StringType(), True), 
                            StructField("Fare",  FloatType(), True), 
                            StructField("Cabin",  StringType(), True),
                               StructField("Embarked",  StringType(), True)]
                          )
df_train = sqlc.read.load(path="data/train.csv", 
                          format="com.databricks.spark.csv", 
                          schema=titanicSchemaTrain,
                          header=True)

df_test = sqlc.read.load(path="data/test.csv", 
                          format="com.databricks.spark.csv", 
                          schema=titanicSchemaTest, header=True)

### Step 2
- Explore the features of your dataset
- You can use DataFrame's ***describe*** method to get summary statistics
    - hint: ***toPandas*** may be useful to ease the manipulation of small dataframes
- Are there any ***NaN*** values in your dataset?
- If so, define value/values to fill these ***NaN*** values
    - hint: ***na*** property of DataFrames provide several methods of handling NA values

### Step 3
- How to handle categorical features?
    - hint: check the Estimators and Transformers
- Assemble all desired features into a Vector using the VectorAssembler Transformer
- Make sure to end up with a DataFrame with two columns: ***Survived*** and ***vFeatures***

In [4]:
age_mean = df_train.describe().toPandas().set_index("summary").loc['mean','Age']
def remove_useless_features(df):
    return df.drop("Cabin")
df_train = remove_useless_features(df_train)
df_test = remove_useless_features(df_test)
from numpy import NaN
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import isnull, isnan, when, count, col

def average_missing_features(df):
    df = df.withColumn("age", when(col('age').isNull(), age_mean).otherwise(col('age'))) 
    df = df.withColumn("Embarked", when(col('Embarked').isNull(), 'C').otherwise(col('Embarked'))) 
    df = df.withColumn("Fare", when(col('Fare').isNull(), 0).otherwise(col('Fare'))) 
    
    
    return df


df_train =     average_missing_features(df_train)
df_test  =     average_missing_features(df_test)


In [7]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
def convert_age(df):
    df = df.withColumn("age", col("age").cast(FloatType()))
    return df
df_train = convert_age(df_train)
df_test = convert_age(df_test)


In [71]:
df_train

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, age: string, SibSp: int, Parch: int, Ticket: string, Fare: float, Embarked: string]

In [72]:
def retrieve_title(name):
    index_c = name.index(',')
    index_point = name.index('.')
    if index_c >= 0 and index_point > 0:
        title = name[index_c+1:index_point].strip()
        return title
    return 'No'


#udf_retr_title = udf(retrieve_title, StringType())
def extract_title(df):
    df = df.withColumn("Name", udf_retr_title('Name') )     
    return df


#extract_title(df_train)    

In [73]:
df_test.describe().toPandas()

,summary,PassengerId,Pclass,Name,Sex,age,SibSp,Parch,Ticket,Fare,Embarked
0,count,418,418,418,418,418,418,418,418,418,418
1,mean,1100.5,2.2655502392344498,None,None,30.154603152261867,0.4473684210526316,0.3923444976076555,223850.98986486485,35.541956141234586,None
2,stddev,120.81045760473994,0.8418375519640503,None,None,12.636665857360075,0.8967595611217135,0.9814288785371694,369523.7764694362,55.86768433838268,None
3,min,892,1,"""Assaf Khalil, Mrs. Mariana (Miriam"""")""""""",female,0.17,0,0,110469,0.0,C
4,max,1309,3,"van Billiard, Master. Walter John",male,9.0,8,9,W.E.P. 5734,512.3292,S


Step 3
How to handle categorical features?
hint: check the Estimators and Transformers
Assemble all desired features into a Vector using the VectorAssembler Transformer
Make sure to end up with a DataFrame with two columns: Survived and vFeatures


In [74]:
from pyspark.ml.feature import StringIndexer
def categorize_df(df):
    indexerS = StringIndexer(inputCol="Sex", outputCol="SexC")
    indexerE = StringIndexer(inputCol="Embarked", outputCol="EmbarkedC")
    
    df = indexerS.fit(df).transform(df)
    df = indexerE.fit(df).transform(df)
    
    df = df.drop("Sex", "Embarked")
    return df

df_train = categorize_df(df_train)
df_test = categorize_df(df_test)

In [75]:
from pyspark.ml.feature import OneHotEncoder
def onehot_df(df):
    oneHotS = OneHotEncoder(inputCol="SexC", outputCol="SexV")
    oneHotE = OneHotEncoder(inputCol="EmbarkedC", outputCol="EmbarkedV")
    
    df = oneHotS.transform(df)
    df = oneHotE.transform(df)
    
    df = df.drop("SexC", "EmbarkedC")
    return df

df_train = onehot_df(df_train)
df_test = onehot_df(df_test)

In [77]:
from pyspark.ml.feature import VectorAssembler

def vectorize(df):

    assembler = VectorAssembler(inputCols = \
                ["Pclass","age", "SibSp", "Parch", "Fare", "SexV", "EmbarkedV"], outputCol = "vFeatures")

    df = assembler.transform(df)
    
    return df

df_train = vectorize(df_train)
df_test = vectorize(df_test)

df_train = df_train['PassengerId','vFeatures','Survived']
df_test = df_test['PassengerId','vFeatures']




In [78]:
df_train.toPandas()

,PassengerId,vFeatures,Survived
0,1,"[3.0, 22.0, 1.0, 0.0, 7.25, 1.0, 1.0, 0.0]",0
1,2,"[1.0, 38.0, 1.0, 0.0, 71.2833023071, 0.0, 0.0,...",1
2,3,"(3.0, 26.0, 0.0, 0.0, 7.92500019073, 0.0, 1.0,...",1
3,4,"[1.0, 35.0, 1.0, 0.0, 53.0999984741, 0.0, 1.0,...",1
4,5,"[3.0, 35.0, 0.0, 0.0, 8.05000019073, 1.0, 1.0,...",0
5,6,"(3.0, 29.6991176605, 0.0, 0.0, 8.45829963684, ...",0
6,7,"[1.0, 54.0, 0.0, 0.0, 51.8624992371, 1.0, 1.0,...",0
7,8,"[3.0, 2.0, 3.0, 1.0, 21.0750007629, 1.0, 1.0, ...",0
8,9,"[3.0, 27.0, 0.0, 2.0, 11.1332998276, 0.0, 1.0,...",1
9,10,"[2.0, 14.0, 1.0, 0.0, 30.0708007812, 0.0, 0.0,...",1


In [58]:
df_test.toPandas()

Py4JJavaError: An error occurred while calling o843.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 59.0 failed 1 times, most recent failure: Lost task 0.0 in stage 59.0 (TID 59, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$3: (struct<Pclass_double_VectorAssembler_4ca9b039333100e7d833:double,age_double_VectorAssembler_4ca9b039333100e7d833:double,SibSp_double_VectorAssembler_4ca9b039333100e7d833:double,Parch_double_VectorAssembler_4ca9b039333100e7d833:double,Fare_double_VectorAssembler_4ca9b039333100e7d833:double,SexV:vector,EmbarkedV:vector>) => vector)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$8$$anon$1.hasNext(WholeStageCodegenExec.scala:395)
	at 

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.util import MLUtils

### INSERT YOUR CODE HERE

### Step 4
- In Step 5, you will apply a normalization Estimator
- BUT, it does not accept feature vectors of the Sparse type
- So, it is neccessary to apply an User Defined Function to make all features vectors of type VectorUDT
- In this step, you only have to replace ***YOUR DATAFRAME*** and ***NEW DATAFRAME*** with your variables

In [79]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.ml.linalg import VectorUDT, Vectors

to_vec = UserDefinedFunction(lambda x: Vectors.dense(x.toArray()), VectorUDT())

df_train = df_train.select("PassengerId", "Survived", to_vec("vFeatures").alias("features"))
df_test = df_test.select("PassengerId", to_vec("vFeatures").alias("features"))


### Step 5
- Apply a normalization Estimator of your choice to the ***features*** vector obtained in Step 4

In [80]:
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=False)

# Compute summary statistics by fitting the StandardScaler
df_train_scalerModel = scaler.fit(df_train)

# Normalize each feature to have unit standard deviation.
df_train = df_train_scalerModel.transform(df_train)

# Compute summary statistics by fitting the StandardScaler
df_test_scalerModel = scaler.fit(df_test)

# Normalize each feature to have unit standard deviation.
df_test = df_test_scalerModel.transform(df_test)



In [82]:
df_train.toPandas()

,PassengerId,Survived,features,scaledFeatures
0,1,0,"[3.0, 22.0, 1.0, 0.0, 7.25, 1.0, 1.0, 0.0]","[3.5882109717, 1.69204539585, 0.906829250318, ..."
1,2,1,"[1.0, 38.0, 1.0, 0.0, 71.2833023071, 0.0, 0.0,...","[1.1960703239, 2.92262386556, 0.906829250318, ..."
2,3,1,"[3.0, 26.0, 0.0, 0.0, 7.92500019073, 0.0, 1.0,...","[3.5882109717, 1.99969001328, 0.0, 0.0, 0.1594..."
3,4,1,"[1.0, 35.0, 1.0, 0.0, 53.0999984741, 0.0, 1.0,...","[1.1960703239, 2.69189040249, 0.906829250318, ..."
4,5,0,"[3.0, 35.0, 0.0, 0.0, 8.05000019073, 1.0, 1.0,...","[3.5882109717, 2.69189040249, 0.0, 0.0, 0.1619..."
5,6,0,"[3.0, 29.6991176605, 0.0, 0.0, 8.45829963684, ...","[3.5882109717, 2.28419342265, 0.0, 0.0, 0.1702..."
6,7,0,"[1.0, 54.0, 0.0, 0.0, 51.8624992371, 1.0, 1.0,...","[1.1960703239, 4.15320233527, 0.0, 0.0, 1.0436..."
7,8,0,"[3.0, 2.0, 3.0, 1.0, 21.0750007629, 1.0, 1.0, ...","[3.5882109717, 0.153822308714, 2.72048775095, ..."
8,9,1,"[3.0, 27.0, 0.0, 2.0, 11.1332998276, 0.0, 1.0,...","[3.5882109717, 2.07660116764, 0.0, 2.481213427..."
9,10,1,"[2.0, 14.0, 1.0, 0.0, 30.0708007812, 0.0, 0.0,...","[2.3921406478, 1.076756161, 0.906829250318, 0...."


### Step 6
- Train a classifier of your choice (for instance, Random Forest) using your dataset of LabeledPoints
- Make predictions for the training data
- Use the Binary Classification Evaluator to evaluate your model on the training data
- How is your model performing? Try to tune its parameters

In [16]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.util import MLUtils
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import SQLTransformer

pipeline = Pipeline(stages=[
    StringIndexer(inputCol="Sex", outputCol="SexC"),
    StringIndexer(inputCol="Embarked", outputCol="EmbarkedC"),
    OneHotEncoder(inputCol="SexC", outputCol="SexV"),
    OneHotEncoder(inputCol="EmbarkedC", outputCol="EmbarkedV"),
    VectorAssembler(inputCols = \
                ["Pclass","age", "SibSp", "Parch", "Fare", "SexV", "EmbarkedV"], outputCol = "features"),
    StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=False)
    SQLTransformer(statement="SELECT PassengerId, Survived, scaledFeatures FROM __THIS__")
])

# Fit the pipeline to training documents.
model = pipeline.fit(df_train)
df_train_2 =  model.transform(df_train)
df_train_3 = df_train_2    

In [17]:
df_train_3.show()

+-----------+--------+------+--------------------+------+---------+-----+-----+----------------+-------+--------+----+---------+-------------+-------------+--------------------+--------------------+
|PassengerId|Survived|Pclass|                Name|   Sex|      age|SibSp|Parch|          Ticket|   Fare|Embarked|SexC|EmbarkedC|         SexV|    EmbarkedV|            features|      scaledFeatures|
+-----------+--------+------+--------------------+------+---------+-----+-----+----------------+-------+--------+----+---------+-------------+-------------+--------------------+--------------------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|     22.0|    1|    0|       A/5 21171|   7.25|       S| 0.0|      0.0|(1,[0],[1.0])|(2,[0],[1.0])|[3.0,22.0,1.0,0.0...|[3.58821097170396...|
|          2|       1|     1|Cumings, Mrs. Joh...|female|     38.0|    1|    0|        PC 17599|71.2833|       C| 1.0|      1.0|    (1,[],[])|(2,[1],[1.0])|[1.0,38.0,1.0,0.0...|[1.19607032390132...|
|    

In [18]:
df_train_3.select('Survived', 'scaledFeatures').toPandas()

,Survived,scaledFeatures
0,0,"[3.5882109717, 1.69204539585, 0.906829250318, ..."
1,1,"[1.1960703239, 2.92262386556, 0.906829250318, ..."
2,1,"(3.5882109717, 1.99969001328, 0.0, 0.0, 0.1594..."
3,1,"[1.1960703239, 2.69189040249, 0.906829250318, ..."
4,0,"[3.5882109717, 2.69189040249, 0.0, 0.0, 0.1619..."
5,0,"(3.5882109717, 2.28419342265, 0.0, 0.0, 0.1702..."
6,0,"[1.1960703239, 4.15320233527, 0.0, 0.0, 1.0436..."
7,0,"[3.5882109717, 0.153822308714, 2.72048775095, ..."
8,1,"[3.5882109717, 2.07660116764, 0.0, 2.481213427..."
9,1,"[2.3921406478, 1.076756161, 0.906829250318, 0...."


In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import RandomForestClassificationModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator

### INSERT YOUR CODE HERE

### Step 7
- Take a look at the test data - use DataFrame's ***createOrReplaceTempView*** method to perform SQL queries over the data
    - hint: check if there are any NULL values in the dataset - if so, handle them
- Apply the transformations to the test data
    - hint: you can use Pipelines to chain several Estimators/Transformers
    - warning: unfortunately, it is not possible to include the UDF from Step 4 in the Pipeline
- Make predictions using the model previously trained and the transformed test data
- Save it as ***submission.csv*** and submit it to Kaggle
- What was your score?

In [ ]:
from pyspark.ml import Pipeline

### INSERT YOUR CODE HERE

In [ ]:
df_predictions = <YOUR PREDICTIONS DATAFRAME>.select("prediction").toPandas().reset_index()
df_predictions['index'] = df_predictions['index'] + 892
df_predictions.columns = ['PassengerId', 'Survived']

df_predictions.to_csv('submission.csv', index=False)

## Result = ???%